# Базовые библиотеки

In [1]:
import pandas as pd
import os
import warnings
warnings.simplefilter('ignore')

In [2]:
import data_extracting
import merging
import visualization

In [3]:
DATA_DIR = 'Airlines'
DATA_EXPORTED = 'data'

# Извлечение данных в формат CSV
Запускать только если ещё нет экспортированных данных, так как некоторые функции могут работать существенное время

In [ ]:
os.mkdir(DATA_EXPORTED)

In [ ]:
data = data_extracting.extractBoardingData(f'{DATA_DIR}/BoardingData.csv')
data.to_csv(f'{DATA_EXPORTED}/BoardingData.csv', index=False)
del data

In [ ]:
data = data_extracting.extractAirlinesData(f'{DATA_DIR}/PointzAggregator-AirlinesData.xml')
data.to_csv(f'{DATA_EXPORTED}/PointzAggregator-AirlinesData.csv', index=False)
del data

In [ ]:
data = data_extracting.extractBoardingPass(f'{DATA_DIR}/YourBoardingPassDotAero.zip', clear_temp = True)
data.to_csv(f'{DATA_EXPORTED}/YourBoardingPassDotAero.csv', index=False)
del data

In [ ]:
data = data_extracting.extractFrequentFlyerForumProfiles(f'{DATA_DIR}/FrequentFlyerForum-Profiles.json')
data[0].to_csv(f'{DATA_EXPORTED}/FrequentFlyerForum-Profiles_Base.csv', index=False)
data[1].to_csv(f'{DATA_EXPORTED}/FrequentFlyerForum-Profiles_Flight.csv', index=False)
data[2].to_csv(f'{DATA_EXPORTED}/FrequentFlyerForum-Profiles_LoyalityProgram.csv', index=False)
del data

In [ ]:
data = data_extracting.extractSirenaExportFixed(f'{DATA_DIR}/Sirena-export-fixed.tab')
data.to_csv(f'{DATA_EXPORTED}/Sirena-export-fixed.csv', index=False)
del data

In [ ]:
data = data_extracting.extractSkyTeamExchange(f'{DATA_DIR}/SkyTeam-Exchange.yaml')
data.to_csv(f'{DATA_EXPORTED}/SkyTeam-Exchange.csv', index=False)
del data

In [ ]:
data = data_extracting.extractSkyteamTimetable(f'{DATA_DIR}/Skyteam_Timetable.pdf')
data.to_csv(f'{DATA_EXPORTED}/Skyteam_Timetable.csv', index=False)
del data

# Обработка данных

## 1. Обработка данных, с паспортами
Слияние, очистка и обработка данных из [этого](data/Sirena-export-fixed.csv) и [этого](data/BoardingData.csv) файлов

In [4]:
df_sirena = pd.read_csv('data/Sirena-export-fixed.csv')

In [5]:
df_boarding = pd.read_csv('data/BoardingData.csv', sep = ';')
df_boarding['PassangerName'] = df_boarding['PassengerLastName'] + ' ' + df_boarding['PassengerFirstName']

In [6]:
# Проверка - нет ли людей, менявших имя, или указавших разные данные
dup1 = df_boarding[['PassengerDocument','PassangerName']].groupby('PassengerDocument')['PassangerName'].nunique()
dup1 = dup1[dup1 > 1].index.tolist()
print(f'Странные паспорта: {dup1}')
if len(dup1) > 0:
    print(df_boarding[df_boarding['PassengerDocument'].isin(dup1)][['PassangerName', 'PassengerDocument', 'PassengerBirthDate']].drop_duplicates())

Странные паспорта: ['8248 013778']
        PassangerName PassengerDocument PassengerBirthDate
83246   GLEBOV GORDEI       8248 013778         12/31/1982
150151    BUROV RAMIL       8248 013778         08/24/1995


In [7]:
data1, docs = merging.mergeDataPasports(df_sirena, df_boarding)

In [8]:
data1['ID'] = "pass_" + data1['PassengerDocument']
data1.rename(columns={'Dest':'To','FlightDate':'Date'}, inplace=True)

In [9]:
print('Соответствие загранпаспортов и паспортов РФ:')
docs

Соответствие загранпаспортов и паспортов РФ:


,pass,PassengerDocument
35,44 2285514,3342 036804
65,78 5386060,2924 413280
78,21 5558588,8508 948406
213,09 8507665,5237 499392
248,71 8104777,0085 524168
...,...,...
154890,48 5701014,5400 482368
154911,13 9652212,0977 338850
155061,27 8836096,9258 183384
155090,18 5258526,8436 085223


## 2. Обработка данных, на основе уникальных номеров программ лояльности

In [10]:
df_airlines = pd.read_csv('data/PointzAggregator-AirlinesData.csv')
df_airlines['uid'] = df_airlines['uid'].apply(str)

In [11]:
df_exchange = pd.read_csv('data/SkyTeam-Exchange.csv')

In [12]:
df_forum_b = pd.read_csv('data/FrequentFlyerForum-Profiles_Base.csv')
df_forum_f = pd.read_csv('data/FrequentFlyerForum-Profiles_Flight.csv')
df_forum_lp = pd.read_csv('data/FrequentFlyerForum-Profiles_LoyalityProgram.csv')
df_forum = (df_forum_b,df_forum_f,df_forum_lp)

In [13]:
data2, strange, ids = merging.mergeLoyality(df_exchange, df_forum, df_airlines)

In [14]:
print('Нехорошие люди, использовавшие чужие программы лояльности:')
strange

Нехорошие люди, использовавшие чужие программы лояльности:


,uid,FFKey
3780,866799462,FB 139888197


## 3. Слияние всего вместе

In [15]:
# Соответствие ников, uid и паспорта
ids = merging.mergeLoyalityIdNickPasports(df_sirena, ids)

In [16]:
# Данные, о полётах из второй части данных
data = pd.merge(data2, ids[['ID','FFKey']], on='FFKey', how='left')[['ID', 'Date','From','To']]
data = data.dropna()

In [17]:
# Объединение всех данных о полётах вместе, для дальнейшей отрисовки
data = pd.concat([data, data1[['ID','Date','From','To']]])

In [18]:
data

,ID,Date,From,To
0,id_438181628,2017-01-01,SVO,CDG
1,id_438181628,2017-01-06,CDG,ZRH
2,id_438181628,2017-01-15,ZRH,AMS
3,id_438181628,2017-02-02,AMS,SXB
4,id_438181628,2017-02-03,SXB,AMS
...,...,...,...,...
155429,pass_3242 441269,2017-01-08,ARH,SVO
155435,pass_5860 040208,2017-02-14,VOG,SVO
155438,pass_4553 277857,2017-02-14,VOG,SVO
155439,pass_0282 596160,2017-05-18,SVO,ROV


# Отрисовка графа полётов

In [ ]:
airports = pd.read_csv('data/Airports.csv')

In [ ]:
data = visualization.get_dataframe_for_work(data, airports)

In [ ]:
visualization.create_flight_graph_app(data)